## 回归简介 - 第1课

#### 放到实际情境中

✅ 回归方法有很多种，选择哪一种取决于你想要得到的答案。如果你想预测某个年龄段的人可能的身高，你会使用 `线性回归`，因为你在寻找一个**数值结果**。如果你想知道某种菜肴是否应该被归类为素食，你是在寻找一个**类别分配**，因此你会使用 `逻辑回归`。稍后你会学习更多关于逻辑回归的内容。试着思考一些你可以从数据中提出的问题，以及哪种方法更适合这些问题。

在本节中，你将使用一个[关于糖尿病的小型数据集](https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html)。假设你想测试一种针对糖尿病患者的治疗方法。机器学习模型可能会帮助你根据变量的组合确定哪些患者对治疗的反应更好。即使是一个非常基础的回归模型，在可视化时也可能显示出一些关于变量的信息，这些信息可以帮助你组织理论上的临床试验。

话不多说，让我们开始这项任务吧！

<p >
   <img src="../../images/encouRage.jpg"
   width="630"/>
   <figcaption>由 @allison_horst 创作的艺术作品</figcaption>

<!--![由 \@allison_horst 创作的艺术作品](../../../../../../translated_images/encouRage.e75d5fe0367fb9136b78104baf4e2032a7622bc42a2bc34c0ad36c294eeb83f5.zh.jpg)<br>由 @allison_horst 创作的艺术作品-->


## 1. 加载工具集

在这个任务中，我们需要以下软件包：

-   `tidyverse`： [tidyverse](https://www.tidyverse.org/) 是一个[由 R 软件包组成的集合](https://www.tidyverse.org/packages)，旨在让数据科学更快速、更简单、更有趣！

-   `tidymodels`： [tidymodels](https://www.tidymodels.org/) 框架是一个[由软件包组成的集合](https://www.tidymodels.org/packages/)，用于建模和机器学习。

你可以通过以下命令安装它们：

`install.packages(c("tidyverse", "tidymodels"))`

下面的脚本会检查你是否拥有完成本模块所需的软件包，并在缺少某些软件包时为你安装它们。


In [2]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))
pacman::p_load(tidyverse, tidymodels)

Loading required package: pacman



现在，让我们加载这些超棒的包并使它们在我们当前的 R 会话中可用。（这只是为了说明，`pacman::p_load()` 已经为您完成了这一操作）


In [ ]:
# load the core Tidyverse packages
library(tidyverse)

# load the core Tidymodels packages
library(tidymodels)


## 2. 糖尿病数据集

在本次练习中，我们将通过对糖尿病数据集进行预测来展示我们的回归技能。[糖尿病数据集](https://www4.stat.ncsu.edu/~boos/var.select/diabetes.rwrite1.txt) 包含 `442 个样本`，数据包括 10 个预测特征变量：`年龄`、`性别`、`身体质量指数`、`平均血压`以及 `六项血清测量值`，还有一个结果变量 `y`：衡量基线后一年疾病进展的定量指标。

|观察数量|442|
|----------------------|:---|
|预测变量数量|前 10 列为数值型预测变量|
|结果/目标|第 11 列为基线后一年疾病进展的定量指标|
|预测变量信息|- 年龄（以年为单位）
||- 性别
||- bmi 身体质量指数
||- bp 平均血压
||- s1 tc，总血清胆固醇
||- s2 ldl，低密度脂蛋白
||- s3 hdl，高密度脂蛋白
||- s4 tch，总胆固醇 / HDL
||- s5 ltg，可能是血清甘油三酯水平的对数值
||- s6 glu，血糖水平|

> 🎓 记住，这是监督学习，我们需要一个名为 'y' 的目标变量。

在使用 R 操作数据之前，您需要将数据导入 R 的内存，或者建立一个 R 可以用来远程访问数据的连接。

> [readr](https://readr.tidyverse.org/) 包是 Tidyverse 的一部分，它提供了一种快速且友好的方式将矩形数据读入 R。

现在，让我们加载来源 URL 提供的糖尿病数据集：<https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html>

此外，我们将使用 `glimpse()` 对数据进行基本检查，并使用 `slice()` 显示前 5 行。

在继续之前，让我们介绍一个您在 R 代码中经常会遇到的东西 🥁🥁：管道操作符 `%>%`

管道操作符 (`%>%`) 按逻辑顺序执行操作，将一个对象传递到函数或表达式中。您可以将管道操作符理解为代码中的“然后”。


In [ ]:
# Import the data set
diabetes <- read_table2(file = "https://www4.stat.ncsu.edu/~boos/var.select/diabetes.rwrite1.txt")


# Get a glimpse and dimensions of the data
glimpse(diabetes)


# Select the first 5 rows of the data
diabetes %>% 
  slice(1:5)

`glimpse()` 命令显示，这个数据集有 442 行和 11 列，所有列的数据类型均为 `double`。

<br>

> `glimpse()` 和 `slice()` 是 [`dplyr`](https://dplyr.tidyverse.org/) 包中的函数。Dplyr 是 Tidyverse 的一部分，它是一种数据操作的语法，提供了一组一致的动词，帮助解决最常见的数据处理问题。

<br>

现在我们已经有了数据，接下来让我们聚焦于一个特征（`bmi`），作为本次练习的目标。为此，我们需要选择所需的列。那么，我们该如何操作呢？

[`dplyr::select()`](https://dplyr.tidyverse.org/reference/select.html) 函数允许我们在数据框中*选择*（并可选地重命名）列。


In [ ]:
# Select predictor feature `bmi` and outcome `y`
diabetes_select <- diabetes %>% 
  select(c(bmi, y))

# Print the first 5 rows
diabetes_select %>% 
  slice(1:10)

## 3. 训练和测试数据

在监督学习中，通常的做法是将数据*分割*成两个子集：一个（通常较大的）用于训练模型的集合，以及一个较小的“保留”集合，用于查看模型的表现如何。

现在我们已经准备好了数据，可以看看机器是否能够帮助我们在这个数据集中找到一个合理的分割方式。我们可以使用 [rsample](https://tidymodels.github.io/rsample/) 包，它是 Tidymodels 框架的一部分，用来创建一个包含数据分割信息的对象，然后使用另外两个 rsample 函数提取生成的训练集和测试集：


In [ ]:
set.seed(2056)
# Split 67% of the data for training and the rest for tesing
diabetes_split <- diabetes_select %>% 
  initial_split(prop = 0.67)

# Extract the resulting train and test sets
diabetes_train <- training(diabetes_split)
diabetes_test <- testing(diabetes_split)

# Print the first 3 rows of the training set
diabetes_train %>% 
  slice(1:10)

## 4. 使用Tidymodels训练线性回归模型

现在我们可以开始训练模型了！

在Tidymodels中，你可以通过`parsnip()`指定模型，主要涉及以下三个概念：

-   模型**类型**区分了不同的模型，例如线性回归、逻辑回归、决策树模型等。

-   模型**模式**包括常见选项如回归和分类；某些模型类型支持这两种模式，而有些仅支持其中一种。

-   模型**引擎**是用于拟合模型的计算工具。通常这些是R包，例如 **`"lm"`** 或 **`"ranger"`**。

这些建模信息会被捕获到一个模型规范中，所以让我们来构建一个吧！


In [ ]:
# Build a linear model specification
lm_spec <- 
  # Type
  linear_reg() %>% 
  # Engine
  set_engine("lm") %>% 
  # Mode
  set_mode("regression")


# Print the model specification
lm_spec

在模型被*指定*之后，可以使用 [`fit()`](https://parsnip.tidymodels.org/reference/fit.html) 函数对模型进行`估计`或`训练`，通常使用公式和一些数据。

`y ~ .` 表示我们将拟合 `y` 作为预测值/目标，由所有预测变量/特征解释，即 `.`（在这个例子中，我们只有一个预测变量：`bmi`）。


In [ ]:
# Build a linear model specification
lm_spec <- linear_reg() %>% 
  set_engine("lm") %>%
  set_mode("regression")


# Train a linear regression model
lm_mod <- lm_spec %>% 
  fit(y ~ ., data = diabetes_train)

# Print the model
lm_mod

从模型输出中，我们可以看到训练过程中学习到的系数。它们表示最佳拟合线的系数，该线使实际变量与预测变量之间的总体误差最小。
<br>

## 5. 在测试集上进行预测

现在我们已经训练了一个模型，可以使用它通过 [parsnip::predict()](https://parsnip.tidymodels.org/reference/predict.model_fit.html) 来预测测试数据集中的疾病进展 y。这将用于绘制数据组之间的分界线。


In [ ]:
# Make predictions for the test set
predictions <- lm_mod %>% 
  predict(new_data = diabetes_test)

# Print out some of the predictions
predictions %>% 
  slice(1:5)

哇哦！💃🕺 我们刚刚训练了一个模型，并用它进行了预测！

在进行预测时，tidymodels 的惯例是始终生成一个带有标准化列名的 tibble/数据框结果。这使得将原始数据和预测结果结合在一个可用的格式中变得非常简单，方便后续操作，例如绘图。

`dplyr::bind_cols()` 可以高效地将多个数据框按列绑定在一起。


In [ ]:
# Combine the predictions and the original test set
results <- diabetes_test %>% 
  bind_cols(predictions)


results %>% 
  slice(1:5)

## 6. 绘制模型结果

现在是时候用视觉化方式来看结果了 📈。我们将创建一个散点图，展示测试集中的所有 `y` 和 `bmi` 值，然后使用预测结果绘制一条线，将模型的数据分组之间连接起来，放在最合适的位置。

R 有多种绘图系统，但 `ggplot2` 是其中最优雅且最灵活的一个。它允许你通过**组合独立组件**来构建图表。


In [ ]:
# Set a theme for the plot
theme_set(theme_light())
# Create a scatter plot
results %>% 
  ggplot(aes(x = bmi)) +
  # Add a scatter plot
  geom_point(aes(y = y), size = 1.6) +
  # Add a line plot
  geom_line(aes(y = .pred), color = "blue", size = 1.5)

✅ 想一想这里发生了什么。一条直线穿过了许多小数据点，但它究竟在做什么呢？你能看出如何利用这条直线来预测一个新的、未见过的数据点应该如何与图表的 y 轴相关联吗？试着用语言描述这个模型的实际用途。

恭喜你！你已经构建了第一个线性回归模型，用它进行了预测，并在图表中展示了结果！



---

**免责声明**：  
本文档使用AI翻译服务[Co-op Translator](https://github.com/Azure/co-op-translator)进行翻译。尽管我们努力确保翻译的准确性，但请注意，自动翻译可能包含错误或不准确之处。原始语言的文档应被视为权威来源。对于关键信息，建议使用专业人工翻译。我们不对因使用此翻译而产生的任何误解或误读承担责任。
